In [1]:
!pip install pyproj

In [1]:
# import libraries
import requests
import json
from pyproj import Transformer

# set up transformer from Lambert72 to lat/lon
transformer = Transformer.from_crs("EPSG:31370", "EPSG:4326")

In [2]:
# endpoint url
url = 'https://loket.onroerenderfgoed.be/archeologie/rapporten/eindverslagen/'

# set headers
headers = {'Accept': 'application/json', 
           'User-Agent': 'python-requests/2.18.1',
           'From': 'm.wansleeben@arch.leidenuniv.nl (EXALT)'  
          }

# set parameters
# https://loket.onroerenderfgoed.be/archeologie/rapporten/eindverslagen?sort=id&per_pagina=50&pagina=1
params = {'per_pagina': 50, # number of results per page
          'sort': 'id',
          'pagina': 1 }   # page 


In [3]:
# Create a requests Session
session = requests.Session()

# Set the header options
session.headers.update(headers)

# Make a request and store the response
try:
    response = session.get(url, params=params)
    response.raise_for_status()                 # Raise error in case of failure 
except requests.exceptions.HTTPError as httpErr: 
    print ("HTTP Error:",httpErr) 
except requests.exceptions.ConnectionError as connErr: 
    print ("Error Connecting:",connErr) 
except requests.exceptions.Timeout as timeOutErr: 
    print ("Timeout Error:",timeOutErr) 
except requests.exceptions.RequestException as reqErr: 
    print ("Something Else:",reqErr)
    
# get json data from request
data = response.json()

In [4]:
# go through projects
for project in data:
    #print(json.dumps(project, indent=4))

    # get more details on project using uri
    response = requests.get(
        project['uri'],
        headers = {'Accept': 'application/json'}
    )
    project_detail = response.json()

    #print(json.dumps(data, sort_keys=True, indent=4))

    # loop through relevant documents, make document.json for each
    for doc in project_detail['bijlagen']:
        if doc['bijlagetype']['id'] == 2: # is verslag / rapport
            #print(doc['bestandsnaam'])
            
            output_document = {}
            output_document['source'] = 'Onroerend Erfgoed'
            output_document['file_name'] = doc['bestandsnaam']
            output_document['file_type'] = 'report'
            if(doc['omschrijving']):
                title = f"{project['onderwerp']} - {doc['omschrijving']}"
            else:
                title = f"{project['onderwerp']}"
            output_document['title'] = title
            creators = []
            for auteur in doc['auteurs']:
                creators.append(auteur['auteur'])
            output_document['creators'] = creators
            output_document['description'] = '' # no descriptions in this data source
            output_document['publisher'] = project['archeoloog']
            output_document['createdAt'] = project['datum_indienen'][:10]
            output_document['identifiers'] = {'uri':project['uri']}
            output_document['language'] = 'Dutch'
            
            #coordinates
            totalX = 0
            totalY = 0
            coordinates = project_detail['archeologierapport']['locatie']['contour']['coordinates'][0][0]
            for coord in coordinates:
                totalX += coord[0]
                totalY += coord[1]
            coordX = totalX / len(coordinates)
            coordY = totalY / len(coordinates)
            lat, lon = transformer.transform(coordX,coordY)
            output_document['coordX'] = coordX
            output_document['coordY'] = coordY
            output_document['location'] = {'lat':lat,'lon':lon}
            
            print(json.dumps(output_document, indent=4))
            
            
            

{
    "source": "Onroerend Erfgoed",
    "file_name": "Alle bijlagen Kortrijk Sint-Jans.pdf",
    "file_type": "report",
    "title": "Vooronderzoek Kortrijk Kortrijk-Sint-Janslaan - Alle bijlages",
    "creators": [
        "Krug, Camille"
    ],
    "description": "",
    "publisher": "BAAC Vlaanderen bvba",
    "createdAt": "2017-01-18",
    "identifiers": {
        "uri": "https://id.erfgoed.net/archeologie/eindverslagen/8"
    },
    "language": "Dutch",
    "coordX": 72644.90144278973,
    "coordY": 168956.86609745503,
    "location": {
        "lat": 50.82589905612942,
        "lon": 3.270806796241695
    }
}
{
    "source": "Onroerend Erfgoed",
    "file_name": "Eindverslag_VvR_Kortrijk Sint-Jans.pdf",
    "file_type": "report",
    "title": "Vooronderzoek Kortrijk Kortrijk-Sint-Janslaan",
    "creators": [
        "Baeyens, Nathalie",
        "Camille, Krug"
    ],
    "description": "",
    "publisher": "BAAC Vlaanderen bvba",
    "createdAt": "2017-01-18",
    "identifiers":

{
    "source": "Onroerend Erfgoed",
    "file_name": "bocho_no6130_compleet.pdf",
    "file_type": "report",
    "title": "Vooronderzoek Bocholt De Bocholter Graven",
    "creators": [
        "Ellenkamp, Reinier"
    ],
    "description": "",
    "publisher": "Janssens, Marlien",
    "createdAt": "2017-11-30",
    "identifiers": {
        "uri": "https://id.erfgoed.net/archeologie/eindverslagen/98"
    },
    "language": "Dutch",
    "coordX": 239435.63851837872,
    "coordY": 209902.55722737347,
    "location": {
        "lat": 51.192162897352134,
        "lon": 5.6481495390140415
    }
}
{
    "source": "Onroerend Erfgoed",
    "file_name": "OAOE092-001-Dendermonde Vlasmarkt.pdf",
    "file_type": "report",
    "title": "Opgraving Dendermonde Toevalsvondstdossier 108 - eindverslag toevalsvondst",
    "creators": [
        "Moens, Jan"
    ],
    "description": "",
    "publisher": "Moens, Jan",
    "createdAt": "2018-02-01",
    "identifiers": {
        "uri": "https://id.erfgoed.n

{
    "source": "Onroerend Erfgoed",
    "file_name": "Deurne_Wim Saerens_eindverslag.pdf",
    "file_type": "report",
    "title": "Opgraving Antwerpen Deurne, Wim Saerensplein - eindverslag",
    "creators": [
        "De Rijck, Anna"
    ],
    "description": "",
    "publisher": "ABO NV",
    "createdAt": "2018-07-25",
    "identifiers": {
        "uri": "https://id.erfgoed.net/archeologie/eindverslagen/204"
    },
    "language": "Dutch",
    "coordX": 157633.6740257672,
    "coordY": 213206.2603810498,
    "location": {
        "lat": 51.22874024872103,
        "lon": 4.47803922648791
    }
}
{
    "source": "Onroerend Erfgoed",
    "file_name": "Diksmuide Grote Markt-eindverslag.pdf",
    "file_type": "report",
    "title": "Vooronderzoek Diksmuide Diksmuide Grote Markt - Eindverslag",
    "creators": [
        "De Gryse, Janiek",
        "Eggermont, Shari"
    ],
    "description": "",
    "publisher": "Ruben Willaert nv",
    "createdAt": "2018-08-01",
    "identifiers": {
   

{
    "source": "Onroerend Erfgoed",
    "file_name": "VEC 70_4190398_Belgie, Brasschaat, Dullingen 46, DO.pdf",
    "file_type": "report",
    "title": "Opgraving Brasschaat Brasschaat, Dullingen 46",
    "creators": [
        "Griffioen, Arthur",
        "Hazen, Peter",
        "Le Cotey, Rick",
        "Vernimmen, Tamara"
    ],
    "description": "",
    "publisher": "VLAAMS ERFGOED CENTRUM bvba (VEC)",
    "createdAt": "2018-11-01",
    "identifiers": {
        "uri": "https://id.erfgoed.net/archeologie/eindverslagen/274"
    },
    "language": "Dutch",
    "coordX": 159277.9676677419,
    "coordY": 223406.07809354836,
    "location": {
        "lat": 51.32039378707993,
        "lon": 4.501841469840251
    }
}
{
    "source": "Onroerend Erfgoed",
    "file_name": "Eindverslag_JCO_PPY_Harelbeke.pdf",
    "file_type": "report",
    "title": "Opgraving Harelbeke Harelbeke Marktplein",
    "creators": [
        "Coenaerts, Jan",
        "Pype, Pedro"
    ],
    "description": "",
    

{
    "source": "Onroerend Erfgoed",
    "file_name": "OAOE106-001.pdf",
    "file_type": "report",
    "title": "Opgraving Roeselare Roeselare-Ventweg Zuid - Eindverslag",
    "creators": [
        "Dewilde, Marc"
    ],
    "description": "",
    "publisher": "Agentschap Onroerend Erfgoed",
    "createdAt": "2018-12-22",
    "identifiers": {
        "uri": "https://id.erfgoed.net/archeologie/eindverslagen/323"
    },
    "language": "Dutch",
    "coordX": 63611.45990569913,
    "coordY": 186277.7413310917,
    "location": {
        "lat": 50.9803113984519,
        "lon": 3.1385415822843945
    }
}
{
    "source": "Onroerend Erfgoed",
    "file_name": "Eindverslag_Dilsen-StokkemNieuwstraat_Rapport.pdf",
    "file_type": "report",
    "title": "Opgraving Dilsen-Stokkem Dilsen Stokkem Nieuwstraat",
    "creators": [
        "Steenhoudt, Michiel",
        "Van Remoorter, Olivier"
    ],
    "description": "",
    "publisher": "BAAC Vlaanderen bvba",
    "createdAt": "2019-01-14",
    "id

{
    "source": "Onroerend Erfgoed",
    "file_name": "D-18.0415 Rapportage.pdf",
    "file_type": "report",
    "title": "Opgraving Knokke-Heist Knokke-Heist Retranchementstraat - Verslag Dendrochronologische waardering",
    "creators": [
        "d, p"
    ],
    "description": "",
    "publisher": "GHENT ARCHAEOLOGICAL TEAM bvba",
    "createdAt": "2019-02-04",
    "identifiers": {
        "uri": "https://id.erfgoed.net/archeologie/eindverslagen/355"
    },
    "language": "Dutch",
    "coordX": 80177.31246120157,
    "coordY": 225530.45409508143,
    "location": {
        "lat": 51.33533516654846,
        "lon": 3.3668786498375014
    }
}
{
    "source": "Onroerend Erfgoed",
    "file_name": "2018F80_Knokke_Retranchementstraat_Eindrapport.pdf",
    "file_type": "report",
    "title": "Opgraving Knokke-Heist Knokke-Heist Retranchementstraat",
    "creators": [
        "Laloo, Pieter",
        "Vergauwe, Ruben"
    ],
    "description": "",
    "publisher": "GHENT ARCHAEOLOGICAL TEA

In [ ]:
transformer = Transformer.from_crs("EPSG:31370", "EPSG:4326")
transformer.transform(108583.4294676975,197026.342385482)